# Activity Selection
The activity selection problem (or, BOW course selection problem) is to select the greatest number of non-overlapping activities, where each activity has a start and an end time.

Contrary to what I said in class, there *is* a greedy algorithm for activity selection that finds the optimal solution. The key is to sort activities by their *end* times, not their *start* times. (More below.)

The *weighted* activity selection problem assigns a different weight to each activity.
Weighted activity selection is not amenable to a greedy algorithm, but it is a candidate for dynamic programming.

Gratifyingly, the bottom-up dynamic programming implementation to weighted activity selection is the same as what I sketched in class. That's fleshed out below, in section II **Dynamic Programming**.

References:
* [Activity selection problem (Wikipedia](https://en.wikipedia.org/wiki/Activity_selection_problem)
* [Dynamic Programming (PPT)](http://www.cs.princeton.edu/~wayne/cs423/lectures/dynamic-programming-4up.pdf), Kevin Wayne, COS 423 Analysis of Algorithms Lectures, Spring 2001
* *Introduction to Algorithms*, CLRS. §16.2 (pp. 415-421)

## I. Activity Selection – the greedy algorithm

In [1]:
from collections import namedtuple

# An activity has a start_time and and an end_time.
Activity = namedtuple('Activity', ['start', 'end'])

# A set of activities (represented as a list) used to demonstrate the algorithms.

activities = [Activity(start, end) for start, end in [(0, 6), (1, 4), (3, 5), (3, 8), (4, 7), (5, 9), (6, 10), (8, 11)]]
activities

[Activity(start=0, end=6),
 Activity(start=1, end=4),
 Activity(start=3, end=5),
 Activity(start=3, end=8),
 Activity(start=4, end=7),
 Activity(start=5, end=9),
 Activity(start=6, end=10),
 Activity(start=8, end=11)]

The greedy activity selection algorithm repeatedly adds the best feasible activity that has the earliest end date.

An activity is *feasible* if it doesn't conflict with (its times don't overlap with) any activity already in the solution.

The algorithm defines the (locally) *best* activity as the activity with the earliest end time. (Not the earliest start time. This is where I went wrong – or right, depending on what point I was trying to make about greedy algorithms. :-)

This algorithm works because the activity with the earliest end time conflicts with the smallest number of following activities (activities to its right). An activity A with an earlier end time could be worse than another activity B if A's start time were also earlier than B's start time, such that A conflicted with a *preceding* activity C that was compatible with B. This can't happen with our greedy left-to-right algorithm: if C had been compatible with the other activities in the solution so far, C would have already been added to the solution by the time the algorithm considered A and B, and A wouldn't have been feasible.

(If you don't follow this, there's another explanation on the Wikipedia page.)

In [2]:
def greedy_activity_selection(activities):
    solution = []
    while True:
        # The earliest possible start is the max end time of the activities in the current solution.
        # Any new activity that starts earlier than this will overlap with an existing activity.
        # (If it didn't, it would have been selected earlier.)
        earliest_possible_start = max((activity.end for activity in solution), default=float('-inf'))
        # (This implementation doesn't handle the case of zero-length activities. Later versions happen to.)
        candidates = [activity
                      for activity in activities
                      if activity not in solution and activity.start >= earliest_possible_start]
        best_candidate = min(candidates, key=lambda activity:activity.end, default=None)
        if not best_candidate:
            break
        solution.append(best_candidate)
    return solution
    
greedy_activity_selection(activities)

[Activity(start=1, end=4), Activity(start=4, end=7), Activity(start=8, end=11)]

The algorithm adds activities to the solution list from earliest activity to latest. This can only increase the value of `earliest_possible_start`, each time to the end date of the just-added activity. Therefore, instead of re-computing the earliest possible start each time, update it for each new activity:

In [3]:
def greedy_activity_selection(activities):
    solution = []
    earliest_possible_start = float('-inf')
    while True:
        candidates = [activity
                      for activity in activities
                      if activity not in solution and activity.start >= earliest_possible_start]
        best_candidate = min(candidates, key=lambda activity:activity.end, default=None)
        if not best_candidate:
            break
        solution.append(best_candidate)
        earliest_possible_start = best_candidate.end
    return solution
    
greedy_activity_selection(activities)

[Activity(start=1, end=4), Activity(start=4, end=7), Activity(start=8, end=11)]

Once `earliest_possible_start` has passed an activity's start time, that activity can never be part of the solution. Below, remove infeasible activities from the candidate set. This reduces the work for subsequent to `min`, and also sets up a later optimization.

In [4]:
def greedy_activity_selection(activities):
    candidates = set(activities)
    solution = []
    earliest_possible_start = float('-inf')
    while True:
        candidates = [activity
                      for activity in candidates
                      if activity not in solution and activity.start >= earliest_possible_start]
        best_candidate = min(candidates, key=lambda activity:activity.end, default=None)
        if not best_candidate:
            break
        solution.append(best_candidate)
        earliest_possible_start = best_candidate.end
    return solution

greedy_activity_selection(activities)

[Activity(start=1, end=4), Activity(start=4, end=7), Activity(start=8, end=11)]

Instead of using `min` to look through the entire candidate set each time, sort them by end time once. Then the best candidate is always the first feasible candidate in the list.

In [5]:
def greedy_activity_selection(activities):
    candidates = sorted(activities, key=lambda a:a.end)  # sort increasing by end time
    solution = []
    earliest_possible_start = float('-inf')
    while True:
        candidates = [activity
                      for activity in candidates
                      if activity not in solution and activity.start >= earliest_possible_start]
        if not candidates:
            break
        best_candidate = candidates[0]
        solution.append(best_candidate)
        earliest_possible_start = best_candidate.end
    return solution
    
greedy_activity_selection(activities)

[Activity(start=1, end=4), Activity(start=4, end=7), Activity(start=8, end=11)]

Reverse the order of filter and select. Instead of filtering the candidates and then selecting the best (earliest end time), select the best candidate and then test whether it passes the filter. If it doesn't, ignore it and move onto the next one.

This code uses a Python list – as an data type that can both be sorted like a list and popped like an array – like an implementation in another language might use a heap. Since this algorithm don't actually use the heap INSERT operation (the activities are only sorted once), an implementation in another language would actually use an array – shown below.

In [6]:
def greedy_activity_selection(activities):
    candidates = sorted(activities, key=lambda a:a.end)
    solution = []
    earliest_possible_start = float('-inf')
    while True:
        if not candidates:
            break
        best_candidate = candidates.pop(0)
        if best_candidate.start >= earliest_possible_start:
            solution.append(best_candidate)
            earliest_possible_start = best_candidate.end
    return solution

greedy_activity_selection(activities)

[Activity(start=1, end=4), Activity(start=4, end=7), Activity(start=8, end=11)]

Now that the `break` is at the top of the loop body, it can be promoted into the loop test:

In [7]:
def greedy_activity_selection(activities):
    candidates = sorted(activities, key=lambda a:a.end)
    solution = []
    earliest_possible_start = float('-inf')
    while candidates:
        best_candidate = candidates.pop(0)
        if best_candidate.start >= earliest_possible_start:
            solution.append(best_candidate)
            earliest_possible_start = best_candidate.end
    return solution
    
greedy_activity_selection(activities)

[Activity(start=1, end=4), Activity(start=4, end=7), Activity(start=8, end=11)]

The first candidate is always selected sorted, so we can unwind that part of the loop instead of using infinity.

In [8]:
def greedy_activity_selection(activities):
    candidates = sorted(activities, key=lambda a:a.end)
    solution = []
    best_candidate = candidates.pop(0)
    solution = [best_candidate]
    earliest_possible_start = best_candidate.end
    while candidates:
        best_candidate = candidates.pop(0)
        if best_candidate.start >= earliest_possible_start:
            solution.append(best_candidate)
            earliest_possible_start = best_candidate.end
    return solution
    
greedy_activity_selection(activities)

[Activity(start=1, end=4), Activity(start=4, end=7), Activity(start=8, end=11)]

`earliest_possible_start` is just the end time of the last activity in the solution set. We don't need a separate variable to track it. (Whether this is a performance win, loss, or neutral depends on your compiler or interpreter.)

In [9]:
def greedy_activity_selection(activities):
    candidates = sorted(activities, key=lambda a:a.end)
    solution = [candidates.pop(0)]
    while candidates:
        best_candidate = candidates.pop(0)
        if best_candidate.start >= solution[-1].end:
            solution.append(best_candidate)
    return solution
    
greedy_activity_selection(activities)

[Activity(start=1, end=4), Activity(start=4, end=7), Activity(start=8, end=11)]

The following implementation iterates over a sorted array, instead of popping elements from our list-used-as-heap. It looks more like a standard textbook solution.

In [10]:
def greedy_activity_selection(activities):
    sorted_activities = sorted(activities, key=lambda a:a.end)
    solution = [sorted_activities[0]]
    for candidate in sorted_activities[1:]:
        if candidate.start >= solution[-1].end:
            solution.append(candidate)
    return solution
    
greedy_activity_selection(activities)

[Activity(start=1, end=4), Activity(start=4, end=7), Activity(start=8, end=11)]

For greater similarity to a textbook solution, use an iteration index instead of iterating directly over the array elements:

In [11]:
def greedy_activity_selection(activities):
    sorted_activities = sorted(activities, key=lambda a:a.end)
    solution = [sorted_activities[0]]
    for i in range(1, len(sorted_activities)):
        candidate = sorted_activities[i]
        if candidate.start >= solution[-1].end:
            solution.append(candidate)
    return solution
    
greedy_activity_selection(activities)

[Activity(start=1, end=4), Activity(start=4, end=7), Activity(start=8, end=11)]

Now we can do some analysis. `sorted` is $O(n \log n)$$^1$, and is called once. All the other operations – `append`, `list[i]`, `>=` for small integers – are $O(1)$. There's on loop, whose body consists of $O(1)$ operations. The complexity order of `greedy_activity_selection` is therefore $O(n \log n + n \cdot 1) = O(n \log n)$.

$^1$ You know this because a library function in a mature language will generally use an efficient algorithms. Or, because you found the [time complexity page](https://wiki.python.org/moin/TimeComplexity) of the Python wiki.

## II. Weighted activity selection – dynamic programming

The *weighted* activity selection algorithm adds a weight to each activity.

The optimal selection is the set of compatible activities that sum to the greatest weight.

[The (un-weighted) activity selection problem in the previous section is an instance of the weighted problem, where all the activities have the same weight.]

This section shows the developent and refinement of a bottom-up dynamic programming algorithm.

The first version doesn't take weights into account, and simply returns the measure of the best selection; it doesn't yet report the activities in the selection.

In [12]:
# `bisect(lst, x)` gives the the greatest index i of sorted list lst such that all x < all elements lst[:i]
from bisect import bisect
from itertools import islice, dropwhile

def dynamic_activity_selection(activities):
    activity_end_times = sorted(set(activity.end for activity in activities))
    # scores[i] holds the measure of the best selection whose activities end before activity_end_times[i + 1].
    # in class, I started `scores` at m[-1]. here, it starts at m[0], so that it fits in an array or list.
    scores = [0] * (1 + len(activity_end_times))
    for activity in sorted(activities, key=lambda a:a.end):
        start_measure_index = bisect(activity_end_times, activity.start)
        end_measure_index = bisect(activity_end_times, activity.end)
        # There are two candidate solutions (corresponding to two subproblems): with and without the current activity.
        # The measure *without* the current activity is the same as the measure at the previously computed end time.
        # The measure *with* the current activity is the the measure at the end time of the most recent activity
        # *that is compatible with the current activity*, plus the weight of the current activity.
        #
        # Since a previously considered candidate could share the same end time, consider the previous value at
        # this position in the candidate set too. Initializing `scores[*]` to 0 insures that it will always be
        # updated (as long as the weights, here 1, are positive -- otherwise, `scores[*]` should be initialized
        # to -infinity.)
        score_without_activity = scores[end_measure_index - 1]
        score_with_activity = scores[start_measure_index] + 1
        best_measure = max(scores[end_measure_index], score_without_activity, score_with_activity)
        scores[end_measure_index] = best_measure
    return scores[-1]

dynamic_activity_selection(activities)

3

To report the activities in the selection, record which solution was selected. This involves inlining the functionality of `max`, in order to record additional information based on which of its arguments is selected.

In [13]:
def dynamic_activity_selection(activities):
    activity_end_times = sorted(set(activity.end for activity in activities))
    scores = [0] * (1 + len(activity_end_times))
    # the latest activity in the optimal solution whose activities end at activity_end_times[i + 1]
    selected_activities = [None] * (1 + len(activity_end_times))
    for activity in sorted(activities, key=lambda a:a.end):
        start_measure_index = bisect(activity_end_times, activity.start)
        end_measure_index = bisect(activity_end_times, activity.end)
        score_without_activity = scores[end_measure_index - 1]
        score_with_activity = scores[start_measure_index] + 1
        if score_without_activity > scores[end_measure_index]:
            scores[end_measure_index] = score_without_activity
            selected_activities[end_measure_index] = selected_activities[end_measure_index - 1]
        if score_with_activity > scores[end_measure_index]:
            scores[end_measure_index] = score_with_activity
            selected_activities[end_measure_index] = activity

    # print the results
    activity = selected_activities[-1]
    while activity:
        print(activity)
        activity = selected_activities[bisect(activity_end_times, activity.start)]

dynamic_activity_selection(activities)

Activity(start=8, end=11)
Activity(start=4, end=7)
Activity(start=1, end=4)


We don't actually want to print the results, we want to return them.

Also, this algorithm collects them from last to first; we'll reverse them into the order that they were collected.

Since the results are a set, this isn't actually important. But this demonstrates a couple of common properties of dynamic algorithms:

* The optiimal solution ends up at the end of the table.
* To reconstruct the actual solution, instead of just its score, walk backwards from the end of the table.
* If the solution is a path or list, this will produce it in reverse order. The implementation can either reverse the solution once it is accumulated, or solve a backwards version of the problem so that the accumulated solution is forwards. (In this case, an implementation that reversed the roles of `start` and `end` would yield activities in order from earliest to latest, without additional work.)

In [14]:
def dynamic_activity_selection(activities):
    activity_end_times = sorted(set(activity.end for activity in activities))
    scores = [0] * (1 + len(activity_end_times))
    # the latest activity in the optimal solution whose activities end at activity_end_times[i + 1]
    selected_activities = [None] * (1 + len(activity_end_times))
    for activity in sorted(activities, key=lambda a:a.end):
        start_measure_index = bisect(activity_end_times, activity.start)
        end_measure_index = bisect(activity_end_times, activity.end)
        score_without_activity = scores[end_measure_index - 1]
        score_with_activity = scores[start_measure_index] + 1
        if score_without_activity > scores[end_measure_index]:
            scores[end_measure_index] = score_without_activity
            selected_activities[end_measure_index] = selected_activities[end_measure_index - 1]
        if score_with_activity > scores[end_measure_index]:
            scores[end_measure_index] = score_with_activity
            selected_activities[end_measure_index] = activity

    # accumulate the results
    solution = []
    activity = selected_activities[-1]
    while activity:
        solution.append(activity)  # <- replaces print
        activity = selected_activities[bisect(activity_end_times, activity.start)]
    solution.reverse()
    return solution

dynamic_activity_selection(activities)

[Activity(start=1, end=4), Activity(start=4, end=7), Activity(start=8, end=11)]

The current algorithm indexes `scores` and `selected_activities` by activity end time, so that each entry can accumulate the best score of all the candidate solutions (any activity that ends at that time; or, no activity).

In this particular problem, this isn't necessary. We can partial solutions by the index of the (sorted) activity, instead. If activities $A_i$ and $B_j$ end at the same time, and `scores[i]` includes the score for $A_i$ (because including $A_i$ accumulated a greater score than excluding it), then `scores[j]` will be set to the best of `scores[i]` and the score from adding $B_j$ to a set that excludes $A_i$.

This strategy makes for larger arrays (proportionally to the number of activities, rather than the number of end times), but simpler logic.

In [15]:
def dynamic_activity_selection(activities):
    activity_end_times = sorted(activity.end for activity in activities)  # <-- now sorts a list instead of a set
    scores = [0] * (1 + len(activity_end_times))
    # the latest activity in the optimal solution whose activities end at activity_end_times[i + 1]
    selected_activities = [None] * (1 + len(activity_end_times))
    for activity in sorted(activities, key=lambda a:a.end):
        start_measure_index = bisect(activity_end_times, activity.start)
        end_measure_index = bisect(activity_end_times, activity.end)
        score_without_activity = scores[end_measure_index - 1]
        score_with_activity = scores[start_measure_index] + 1
        # print(end_measure_index)
        if score_without_activity > scores[end_measure_index]:
            scores[end_measure_index] = score_without_activity
            selected_activities[end_measure_index] = selected_activities[end_measure_index - 1]
        if score_with_activity > scores[end_measure_index]:
            scores[end_measure_index] = scores[end_measure_index]
            selected_activities[end_measure_index] = activity

    # accumulate the results
    solution = []
    activity = selected_activities[-1]
    while activity:
        solution.append(activity)  # <- replaces print
        activity = selected_activities[bisect(activity_end_times, activity.start)]
    solution.reverse()
    return solution

dynamic_activity_selection(activities)

[Activity(start=3, end=8), Activity(start=8, end=11)]

`end_measure_index` now strictly increments over the course of the loop (as uncommenting the print statement above will show), so we can do this instead of calling `bisect` to find it each time:

In [16]:
def dynamic_activity_selection(activities):
    activity_end_times = sorted(activity.end for activity in activities)  # <-- now sorts a list instead of a set
    scores = [0] * (1 + len(activity_end_times))
    # the latest activity in the optimal solution whose activities end at activity_end_times[i + 1]
    selected_activities = [None] * (1 + len(activity_end_times))
    end_measure_index = 0
    for activity in sorted(activities, key=lambda a:a.end):
        start_measure_index = bisect(activity_end_times, activity.start)
        end_measure_index += 1  # <- replaces call to bisect
        score_without_activity = scores[end_measure_index - 1]
        score_with_activity = scores[start_measure_index] + 1
        if score_without_activity > score_with_activity:
            scores[end_measure_index] = score_without_activity
            selected_activities[end_measure_index] = selected_activities[end_measure_index - 1]
        else:
            scores[end_measure_index] = score_with_activity
            selected_activities[end_measure_index] = activity

    # accumulate the results
    solution = []
    activity = selected_activities[-1]
    while activity:
        solution.append(activity)  # <- replaces print
        activity = selected_activities[bisect(activity_end_times, activity.start)]
    solution.reverse()
    return solution

dynamic_activity_selection(activities)

[Activity(start=1, end=4), Activity(start=4, end=7), Activity(start=8, end=11)]

Or replace the `scores` and `selected_activities` arrays by lists, and ditch `end_measure_index` entirely:

In [17]:
def dynamic_activity_selection(activities):
    activity_end_times = sorted(activity.end for activity in activities)  # <-- now sorts a list instead of a set
    scores = [0]
    # the latest activity in the optimal solution whose activities end at activity_end_times[i + 1]
    selected_activities = [None]
    for activity in sorted(activities, key=lambda a:a.end):
        start_measure_index = bisect(activity_end_times, activity.start)
        score_without_activity = scores[-1]
        score_with_activity = scores[start_measure_index] + 1
        if score_without_activity > score_with_activity:
            scores.append(score_without_activity)
            selected_activities.append(selected_activities[-1])
        else:
            scores.append(score_with_activity)
            selected_activities.append(activity)

    # accumulate the results
    solution = []
    activity = selected_activities[-1]
    while activity:
        solution.append(activity)  # <- replaces print
        activity = selected_activities[bisect(activity_end_times, activity.start)]
    solution.reverse()
    return solution

dynamic_activity_selection(activities)

[Activity(start=1, end=4), Activity(start=4, end=7), Activity(start=8, end=11)]

Adding activity weights requires replacing the `1` by an activity weight. Here, we'll just use the duration of the activity. This optimizes for the greatest amount of scheduled time, instead of the greatest number of activities.

In [18]:
def dynamic_activity_selection(activities):
    activity_end_times = sorted(activity.end for activity in activities)  # <-- now sorts a list instead of a set
    scores = [0]
    # the latest activity in the optimal solution whose activities end at activity_end_times[i + 1]
    selected_activities = [None]
    for activity in sorted(activities, key=lambda a:a.end):
        start_measure_index = bisect(activity_end_times, activity.start)
        score_without_activity = scores[-1]
        score_with_activity = scores[start_measure_index] + (activity.end - activity.start)
        if score_without_activity > score_with_activity:
            scores.append(score_without_activity)
            selected_activities.append(selected_activities[-1])
        else:
            scores.append(score_with_activity)
            selected_activities.append(activity)

    # accumulate the results
    solution = []
    activity = selected_activities[-1]
    while activity:
        solution.append(activity)  # <- replaces print
        activity = selected_activities[bisect(activity_end_times, activity.start)]
    solution.reverse()
    return solution

dynamic_activity_selection(activities)

[Activity(start=0, end=6), Activity(start=6, end=10)]

The algorithm is $O(n \log n)$. First, the calls to `sorted` outside the loop are each $O(n \log n)$. (We could combine those into a single sort; see below.) Second, the algorithm iterates over all $n$ activities, and calls `bisect` ($O(\log n)$) within each iteration, for another term of $O(n \log n)$.

We can (and probably would) eliminate the call to `bisect` inside the loop. This doesn't affect the asymptotic complexity, but it does affect the constant terms.